# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error

from tqdm import tqdm
import time
import joblib

In [7]:
X_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/214/X_train.npy', allow_pickle=True)
y_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/214/y_train.npy', allow_pickle=True)

X_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/214/X_test.npy', allow_pickle=True)
y_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/214/y_test.npy', allow_pickle=True)

#X_val = np.load('data/x_val.npy')
#y_val = np.load('data/y_val.npy')

In [56]:
X_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/215/X_train.npy', allow_pickle=True)
y_train = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/215/y_train.npy', allow_pickle=True)

X_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/215/X_test.npy', allow_pickle=True)
y_test = np.load('/home/jovyan/ssl-ids/dataset/preprocessed/CIC-IDS-2018/215/y_test.npy', allow_pickle=True)

In [29]:
#X_train = np.concatenate((X_train, X_val), axis=0)
#y_train = np.concatenate((y_train, y_val), axis=0)

# Playground

In [57]:
np.unique(y_train, return_counts=True)

(array(['Benign', 'DoS attacks-GoldenEye', 'DoS attacks-Slowloris'],
       dtype=object),
 array([691545,  29141,   7697]))

In [58]:
normal_data = X_train[y_train == 'Benign'] # normal class

# Helpers

In [59]:
def evaluate_model(model, y_test, y_pred):
    positive_class = 'normal'
    
    accuracy = accuracy_score(y_test, y_pred)
    
    precision = precision_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    recall = recall_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    f1 = f1_score(y_test, y_pred, pos_label=positive_class, zero_division=True)
    
    metrics = {
        'model': [model],
        'accuracy': [accuracy],
        'precision': [precision],
        'recall': [recall],
        'f1': [f1],
    }
    
    return metrics

# OCSVM

In [60]:
from sklearn.svm import OneClassSVM

In [61]:
ocsvm = OneClassSVM(kernel='rbf', gamma='auto')

batch_size = 100
n_batches = int(np.ceil(len(normal_data) / batch_size))

for i in tqdm(range(n_batches), desc="Training OCSVM"):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(normal_data))
    batch_data = normal_data[start:end]
    
    if i == 0:
        ocsvm.fit(batch_data)
    else:
        ocsvm.fit(np.vstack((ocsvm.support_vectors_, batch_data)))

Training OCSVM: 100%|██████████| 6916/6916 [03:42<00:00, 31.13it/s]


In [ ]:
ocsvm = OneClassSVM(kernel='linear', gamma='auto')

start_time = time.time()

ocsvm.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

In [62]:
y_pred_test_ocsvm = ocsvm.predict(X_test)
y_pred_test_ocsvm = np.where(y_pred_test_ocsvm == 1, 'normal', 'anomaly')
y_test_converted_ocsvm = np.where(y_test == 'Benign', 'normal', 'anomaly')

In [63]:
evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)

{'model': ['ocsvm'],
 'accuracy': [0.9561823608241368],
 'precision': [0.9576264784598117],
 'recall': [0.9980093235636482],
 'f1': [0.9774009588139565]}

In [64]:
ocsvm_df = pd.DataFrame(evaluate_model("ocsvm", y_test_converted_ocsvm, y_pred_test_ocsvm)) 
ocsvm_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.956182,0.957626,0.998009,0.977401


# SGDOCSVM

In [65]:
from sklearn.linear_model import SGDOneClassSVM

In [66]:
sgdocsvm = SGDOneClassSVM()

start_time = time.time()

sgdocsvm.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 1.5478932857513428


In [67]:
y_pred_test_sgdocsvm = sgdocsvm.predict(X_test)
y_pred_test_sgdocsvm = np.where(y_pred_test_sgdocsvm == 1, 'normal', 'anomaly')
y_test_converted_sgdocsvm = np.where(y_test == 'Benign', 'normal', 'anomaly')

In [68]:
evaluate_model("sgdocsvm", y_test_converted_sgdocsvm, y_pred_test_sgdocsvm)

{'model': ['sgdocsvm'],
 'accuracy': [0.6474727574626666],
 'precision': [1.0],
 'recall': [0.6286994809620479],
 'f1': [0.772026378482892]}

In [69]:
sgdocsvm_df = pd.DataFrame(evaluate_model("sgdocsvm", y_test_converted_sgdocsvm, y_pred_test_sgdocsvm)) 
sgdocsvm_df

,model,accuracy,precision,recall,f1
0,sgdocsvm,0.647473,1.0,0.628699,0.772026


# LOF

In [70]:
from sklearn.neighbors import LocalOutlierFactor

In [71]:
start_time = time.time()

lof = LocalOutlierFactor(n_neighbors=50, contamination=0.001, novelty=True)
lof.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 131.53099656105042


In [72]:
y_pred_test_lof = lof.predict(X_test)
y_pred_test_lof = np.where(y_pred_test_lof == 1, 'normal', 'anomaly')
y_test_converted_lof = np.where(y_test == 'Benign', 'normal', 'anomaly')

In [73]:
evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)

{'model': ['lof'],
 'accuracy': [0.9486009001649769],
 'precision': [0.9493967147941136],
 'recall': [0.9991171293770982],
 'f1': [0.9736225617925808]}

In [74]:
lof_df = pd.DataFrame(evaluate_model("lof", y_test_converted_lof, y_pred_test_lof)) 
lof_df

,model,accuracy,precision,recall,f1
0,lof,0.948601,0.949397,0.999117,0.973623


# IF

In [75]:
from sklearn.ensemble import IsolationForest

In [76]:
start_time = time.time()

iforest = IsolationForest(n_estimators=200, max_samples=128, contamination=0.001, random_state=21)
iforest.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 134.4211220741272


In [77]:
y_pred_test_if = iforest.predict(X_test)
y_pred_test_if = np.where(y_pred_test_if == 1, 'normal', 'anomaly')
y_test_converted_if = np.where(y_test == "Benign", 'normal', 'anomaly')

In [78]:
evaluate_model("if", y_test_converted_if, y_pred_test_if)

{'model': ['if'],
 'accuracy': [0.9486222563921858],
 'precision': [0.9493977963728853],
 'recall': [0.9991396228961531],
 'f1': [0.9736338105349681]}

In [79]:
if_df = pd.DataFrame(evaluate_model("if", y_test_converted_if, y_pred_test_if)) 
if_df

,model,accuracy,precision,recall,f1
0,if,0.948622,0.949398,0.99914,0.973634


# PCA Reconstruction

In [80]:
from sklearn.decomposition import PCA

In [81]:
start_time = time.time()

pca = PCA(n_components=10)
pca.fit(normal_data)

training_time = time.time() - start_time
print(f"Training Time: {training_time}")

Training Time: 1.8851401805877686


In [82]:
normal_data_pca = pca.transform(normal_data)
normal_data_reconstructed = pca.inverse_transform(normal_data_pca)

train_reconstruction_error = np.mean((normal_data - normal_data_reconstructed) ** 2, axis=1)

In [83]:
threshold = np.percentile(train_reconstruction_error, 95)

In [84]:
X_test_pca = pca.transform(X_test)
X_test_reconstructed = pca.inverse_transform(X_test_pca)

test_reconstruction_error = np.mean((X_test - X_test_reconstructed) ** 2, axis=1)

In [85]:
test_reconstruction_error

array([0.04170892, 0.43751703, 0.06272378, ..., 0.10418225, 0.00406046,
       0.04805483])

In [86]:
y_pred_test_pca = (test_reconstruction_error > threshold).astype(int)
y_pred_test_pca = np.where(y_pred_test_pca == 1, 'normal', 'anomaly')
y_test_converted_pca = np.where(y_test == "Benign", 'normal', 'anomaly')

In [87]:
evaluate_model("pca", y_test_converted_pca, y_pred_test_pca)

{'model': ['pca'],
 'accuracy': [0.08872978499618257],
 'precision': [0.8334110624008955],
 'recall': [0.05024489818870938],
 'f1': [0.09477592150623176]}

In [88]:
pca_df = pd.DataFrame(evaluate_model("pca", y_test_converted_pca, y_pred_test_pca)) 
pca_df

,model,accuracy,precision,recall,f1
0,pca,0.08873,0.833411,0.050245,0.094776


# Results

In [54]:
results_df = pd.concat([ocsvm_df, sgdocsvm_df, lof_df, if_df, pca_df], ignore_index=True)
results_df.to_csv('/home/jovyan/ssl-ids/result/CIC-IDS-2018/results_cic_ids_2018_214.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.694825,0.675729,0.998056,0.805856
1,sgdocsvm,0.696223,1.000000,0.521309,0.685343
2,lof,0.723614,0.696872,0.999036,0.821035
3,if,0.633981,0.634373,0.999028,0.775996
4,pca,0.335480,0.339127,0.049698,0.086692


In [89]:
results_df = pd.concat([ocsvm_df, sgdocsvm_df, lof_df, if_df, pca_df], ignore_index=True)
results_df.to_csv('/home/jovyan/ssl-ids/result/CIC-IDS-2018/results_cic_ids_2018_215.csv', index=False)

results_df

,model,accuracy,precision,recall,f1
0,ocsvm,0.956182,0.957626,0.998009,0.977401
1,sgdocsvm,0.647473,1.000000,0.628699,0.772026
2,lof,0.948601,0.949397,0.999117,0.973623
3,if,0.948622,0.949398,0.999140,0.973634
4,pca,0.088730,0.833411,0.050245,0.094776


In [55]:
joblib.dump(ocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/ocsvm.pkl')
joblib.dump(sgdocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/sgdocsvm.pkl')
joblib.dump(lof, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/lof.pkl')
joblib.dump(iforest, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/iforest.pkl')
joblib.dump(pca, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/pca.pkl')

['/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/pca.pkl']

In [90]:
joblib.dump(ocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/215/ocsvm.pkl')
joblib.dump(sgdocsvm, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/214/sgdocsvm.pkl')
joblib.dump(lof, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/215/lof.pkl')
joblib.dump(iforest, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/215/iforest.pkl')
joblib.dump(pca, '/home/jovyan/ssl-ids/model/CIC-IDS-2018/215/pca.pkl')

['/home/jovyan/ssl-ids/model/CIC-IDS-2018/215/pca.pkl']